# Inference of pretrained model

In [1]:
# system imports
import os

# additional imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler

import torch

# internal imports
from utils import preproces, CoughNet

## Path to test file

In [2]:
fn_wav = 'data/test.wav' # positive example

## Inference

In [4]:
# load model from checkpoint
loaded_checkpoint = torch.load('checkpoints/checkpoint.pth')
hparams = loaded_checkpoint['hparams']
scaler = loaded_checkpoint['scaler']
encoder = loaded_checkpoint['encoder']

model = CoughNet(len(hparams['features']))
model.eval()
model.load_state_dict(loaded_checkpoint['model_state'])

# create input features
df_features = pd.DataFrame(columns=hparams['features'])

# Option 1: Using pd.concat
df_features = pd.concat([df_features, pd.DataFrame([preproces(fn_wav)])], ignore_index=True)

# Option 2: Using loc (alternative approach)
# df_features.loc[len(df_features)] = preproces(fn_wav)

X = np.array(df_features[hparams['features']], dtype=np.float32)
X = torch.Tensor(scaler.transform(X))

outputs = torch.softmax(model(X), 1)
predictions = torch.argmax(outputs.data, 1)

# print result
print(f'model outputs {outputs[0].detach().numpy()} which predicts the class {encoder.classes_[predictions]}!')

/var/folders/js/4gyf2ycn0psfjp__7974j73h0000gn/T/ipykernel_45067/850630829.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_checkpoint = torch.load('checkpoints/ch

model outputs [0.9947754 0.0052246] which predicts the class covid!


/var/folders/js/4gyf2ycn0psfjp__7974j73h0000gn/T/ipykernel_45067/850630829.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame([preproces(fn_wav)])], ignore_index=True)
